# This model using modified similarity (similarity2)

In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt


from torch_geometric.datasets import TUDataset
from preprocessing import data_transformation
from similarity import calculate_similarity_matrix

from model import GCN
import copy

In [2]:
dataset = TUDataset(root='datasets/', name='MUTAG')
torch.manual_seed(1234)
dataset = dataset.shuffle()

## Preprocessing

#### Split: Train test validation

```train_dataset```: for training model<br/>
```val_dataset```: evaluate model for hyperparameter tunning<br/>
```test_dataset```: testing model after complete training<br/>

In [3]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

In [4]:
tr, ts, vl = 0.8, 0.1, 0.1
dslen = len(dataset)
tri = round(tr*dslen)
tsi = round((tr+ts)*dslen)
train_dataset = dataset[:tri]
test_dataset = dataset[tri:tsi]
val_dataset = dataset[tsi:]

In [5]:
dataset.y

tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])

In [6]:
print(len(train_dataset))
train_dataset.y

150


tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1])

In [7]:
len(test_dataset)
test_dataset.y

tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0])

In [8]:
len(val_dataset)
val_dataset.y

tensor([0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])

#### Batching

In [9]:
# paper 128
batch_size = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Building Model

In [36]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphSAGE
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import GINConv
from torch_geometric.nn import GATConv
# from torch_geometric.nn import GINConv
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool
import torch.nn.functional as F

In [43]:
class Base(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels):
        super(Base, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        self.conv1 = SAGEConv(in_channels=dataset.num_node_features, out_channels=hidden_channels)
        self.conv2 = SAGEConv(in_channels=hidden_channels, out_channels=hidden_channels)
                
        # classification layer        
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Embed original
        embedding = self.conv1(x, edge_index)
        embedding = embedding.relu()
        embedding = self.conv2(embedding, edge_index)
        embedding = embedding.relu()
        # subgraph_embedding = subgraph_embedding.relu()
        
        embedding = global_mean_pool(embedding, batch)
        h = self.lin(embedding)
        h = F.relu(h)
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.lin2(h)
        
        return embedding, h

SyntaxError: positional argument follows keyword argument (1253421457.py, line 8)

#### Train

In [38]:
def train_base(model, loader, experiment_mode=False):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    model.train()
    total_loss = 0
    
    for data in loader:
        optimizer.zero_grad()
        if experiment_mode:
            emb, h, S, communities, sub_emb = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        loss = criterion(h, data.y)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    return loss / len(loader)

@torch.no_grad()
def test_base(model, loader, experiment_mode=False):
    model.eval()
    correct = 0
    for data in loader:
        if experiment_mode:
            emb, h, S, communities, sub_emb = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        pred = h.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct/len(loader.dataset)

In [39]:
def baseTrain(train_loader, val_loader, test_loader, epoch = 10, fold=0):
    num_hidden_layer = 128
    base = Base(dataset, num_hidden_layer)
    early_stopping_patience = 20
    best_val_score = -float("inf")
    epochs_without_improvement = 0
    best_state = None
    
    # Train
    for _ in range(epoch):
        
        loss = round(train_base(base, train_loader).item(), 5)
        train_acc = round(test_base(base, train_loader), 5)
        val_acc = round(test_base(base, val_loader), 5)
        
        
        print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; val_acc: {val_acc}; test: {round(test_base(base, test_loader), 2)}')

        if (val_acc > best_val_score):
            best_val_score = val_acc
            epochs_without_improvement = 0
            
            print('best found, save model')
            # save model
            torch.save(base.state_dict(), "model-history/"+str(fold)+".base_best_model-graphsage_data-mutag.pth")
            best_state = copy.deepcopy(base.state_dict())
        else:
            epochs_without_improvement += 1
            if (epochs_without_improvement >= early_stopping_patience):
                print('early stop triggered')
                break
                
            
    # Test
    # test = test_base(best, test_loader)
    # print(f'Accuracy: {test}')
    
    # Create a new instance of the model for testing
    best_model = Base(dataset, num_hidden_layer)
    best_model.load_state_dict(best_state)

    # Test
    test = test_base(best_model, test_loader)
    print(f'Accuracy: {test}')


#### Cross validation 10

In [40]:
from sklearn.model_selection import KFold

In [41]:
train_dataset = dataset[:round(len(dataset) * 0.8)]
test_dataset = dataset[round(len(dataset) * 0.8):]
print(train_dataset.y)
print(len(train_dataset.y))
print(test_dataset.y)

tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1])
150
tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])


In [42]:
# 
train_dataset
test_dataset
k = 10
batch_size = 128

splits = KFold(n_splits=k,shuffle=True,random_state=42)
k_counter = 0
fold_logs = {}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(train_dataset)))):
    print(f'Fold {fold}/{k}')
    
    fold_train = []
    for key in train_idx:
        fold_train.append(train_dataset[key])

    fold_val = [] 
    for key in val_idx:
        fold_val.append(train_dataset[key])

    tr = DataLoader(fold_train, batch_size=batch_size, shuffle=True)
    vd = DataLoader(fold_val, batch_size=batch_size, shuffle=True)
    ts = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
    print("=== Base model vs Experiment ===")
    print("=== Base model ===")
    fold_logs[fold] = baseTrain(tr, vd, ts, 50, fold=fold)
    k_counter += 1

Fold 0/10
=== Base model vs Experiment ===
=== Base model ===


TypeError: __init__() missing 1 required positional argument: 'out_channels'

In [26]:
print(fold_logs)

{0: None, 1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None}


In [80]:
e = Experiment(dataset=dataset, hidden_channels=128)
e.load_state_dict(torch.load("model-history/GIN-MUTAG/6.experiment_best_model-gin_data-mutag.pth"))

<All keys matched successfully>